In [ ]:
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd

In [ ]:
# Select Neural Network Models


## CHOOSE INITIAL STATE -- WILL APPLY TO ALL MODELS
initial_state = np.array([[0],[0]])

## PLOTTING X AXIS LIMITS
x_min = 0
x_max = 40

## TOLERANCE FOR SETTLING TIME CALCULATION
tolerance_percentage = 0.01

In [ ]:
# Circuit Parameters
R1 = 8.9
C1 = 220e-6
L1 = 1e-4
v_i = 12
Ts = 2e-5

# Continuous-Time State Space Equations
Ac = np.array([[0, -1/L1], [1/C1, -1/(R1*C1)]])
Bc = np.array([[1/L1], [0]])
Cc = np.eye(Ac.shape[0])

# Discrete-Time State Space Equations
Ad, Bd, _, _, _ = scipy.signal.cont2discrete((Ac, Bc, Cc, 0), Ts)
inv_Ad = np.linalg.inv(Ad)

In [ ]:
def predict_and_store_history(model_path, state_history, prediction_history):
    loaded_model = tf.keras.models.load_model(model_path)

    state = initial_state
    state_old = initial_state

    predictions = np.zeros((1, 1))
    state_history.append(state.tolist())
    prediction_history.append(predictions.tolist())

    for i in range(1, len(time_steps)):
        input_data = np.hstack((state.reshape(1, -1), state_old.reshape(1, -1)))
        predictions = loaded_model.predict(input_data, verbose=0)
        
        state_old = state
        state = np.dot(Ad, state) + np.dot(Bd, predictions)

        state_history.append(state.tolist())
        prediction_history.append(predictions.tolist())

    state_history = np.array(state_history)
    prediction_history = np.array(prediction_history).reshape(-1, 1)
    
    return state_history, prediction_history

# state_history_PH2 = []
# prediction_history_PH2 = []
# state_history_PH2, prediction_history_PH2 = predict_and_store_history(PH2, state_history_PH2, prediction_history_PH2)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# PLOT 1: Closed Loop Simulation with Prediction Horizon of 2
axes[0].plot(time_steps, state_history_PH2[:, 0], label='State 1: i_L [A]')
axes[0].plot(time_steps, state_history_PH2[:, 1], label='State 2: v_C [V]')
axes[0].step(time_steps, prediction_history_PH5, label='Vin [V]', linestyle='--')
axes[0].set_xlabel('Iterations')
axes[0].set_ylabel('Value')
axes[0].set_title('State Evolution and NN_PH2 Control Actions')
axes[0].legend()
axes[0].grid(True, which='both', linestyle='--', linewidth=0.5)
axes[0].minorticks_on()
axes[0].set_xlim(x_min, x_max)

plt.tight_layout()
plt.show()

In [ ]:
def compute_settling_time(state_history, time_steps, tolerance_percentage, Ts):
    final_value_voltage = state_history[-1, 1]
    tolerance = tolerance_percentage * abs(final_value_voltage)
    settled = np.abs(state_history[:, 1] - final_value_voltage) <= tolerance

    for i in range(len(settled)):
        if all(settled[i:]):
            settling_time_index = i
            settling_time_seconds = time_steps[i] * Ts
            break
    else:
        settling_time_index = settling_time_seconds = None

    return settling_time_index, settling_time_seconds, final_value_voltage, state_history[-1, 0]

time_results = [
    compute_settling_time(state_history, time_steps, tolerance_percentage, Ts)
    for state_history in [
        state_history_PH2
    ]
]

data = {
    "Prediction Horizon": ["2", "5", "10", "10F", "10F2","50", "100", "PH10F2_Mini"],
    "Settling Time [# Iterations]": [res[0] for res in time_results],
    "Settling Time [s]": [res[1] for res in time_results],
    "Final Voltage [V]": [res[2].item() for res in time_results],
    "Final Current [A]": [res[3].item() for res in time_results],
}

df = pd.DataFrame(data)
df_style = df.style.set_properties(**{'text-align': 'center'})
df_style = df_style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
display(df_style)